# Tutorial: Model Selection

In the [previous tutorial](model_evaluation.ipynb), we evaluated the "goodness of fit" of two different models on a data set representing the distribution of centering offsets in a galaxy cluster sample. Here we will continue by doing two styles of quantitative test to compare the two models, with the goal of deciding whether the improved goodness of fit actually justifies the additional complexity of the second model. Specifically, we will compute:
1. The Deviance Information Criterion, one of several possible information criteria, which has the advantages of being relatively simple and having a straightforward Bayesian interpretation.
2. The Bayesian Evidence, a more principled but more complex approach fully in the Bayesian framework.

First, we'll restore everything from the last notebook, including the model class definitions and instances, and the Markov chains they hold.

In [ ]:
exec(open('tbc.py').read()) # define TBC and TBC_above
import dill

# may need to change the load path
TBC() # dill.load_session('../ignore/model_evaluation.db')

exec(open('tbc.py').read()) # (re-)define TBC and TBC_above

In [ ]:
# additional imports go here
from scipy.special import logsumexp
import dynesty
from dynesty import plotting as dyplot

## 1. Calculate the DIC for each model

Recall that the Deviance Information Criterion is given by:

$\mathrm{DIC} = \langle D(\theta) \rangle + p_D; \quad p_D = \langle D(\theta) \rangle - D(\langle\theta\rangle)$

where $\theta$ are the parameters of a model, the deviance $D(\theta)=-2\log P(\mathrm{data}|\theta)$, and averages $\langle\rangle$ are over the posterior distribution of $\theta$.

Write this function, and execute it for the simple model. 

In [ ]:
def DIC(Model):
    """
    Compute the Deviance Information Criterion for the given model.
    (In a less pedagogical world, this would logically be a method of the base Model class.)
    """
    # Compute the deviance D for each sample
    D = -2.0*np.array([ Model.log_likelihood(*params) for params in Model.samples ])
    #pD = 
    #DIC = 
    #return DIC, pD

TBC_above(1)

Compute the DIC for each model.

In [ ]:
DIC1, pD1 = DIC(Model1)
print('Model 1:')
print("Effective number of fitted parameters =", pD1)
print("DIC =", DIC1)

**Self-check:** For Model 1 (the exponential), I get $p_D \approx 0.9$ and DIC $\approx 1668$. As with anything else computed from chains, there will be some stochasticity to the values you compute.

In [ ]:
DIC2, pD2 = DIC(Model2)
print('Model 2:')
print("Effective number of fitted parameters =", pD2)
print("DIC =", DIC2)

Do your values of $p_D$ make intuitive sense?

> _TBC_

Now, to interpret this, we compare the reduction (hopefully) in the DIC of Model 2 compared with Model 1 to the Jeffreys scale (see the notes). By this metric, is your second model better at explaining the data than the exponential model?

In [ ]:
DIC1 - DIC2

> _TBC: your answer_

## 2. Compute the evidence by Monte Carlo integration

To do this, note that

$p(\mathrm{data}|H)=\int d\theta \, p(\mathrm{data}|\theta,H) \, p(\theta|H)$

can be approximated by an average over samples from the prior:

$p(\mathrm{data}|H) \approx \frac{1}{m}\sum_{k=1}^m p(\mathrm{data}|\theta_k,H)$, with $\theta_k\sim p(\theta|H)$.

This estimate is better than trying to use samples from the posterior to calculate the evidence, if only because it's unbiased. But in general, and especially for large-dimensional parameter spaces, it is very inefficient (because the likelihood typically is large in only a small fraction of the prior volume). Still, let's give it a try.

Write a function to draw a large number of samples from the prior and use them to calculate the evidence. To avoid numerical over/underflows, use the special `scipy` function `logsumexp` (which we imported directly, way at the top of the notebook) to do the sum. As the name implies, this function is equivalent to `log(sum(exp(...)))`, but is more numerically stable.

In [ ]:
def log_evidence(Model, N=1000):
    """
    Compute the log evidence for the model using N samples from the prior
    """
    TBC()
    
TBC_above()

Do a quick test to check for NaNs:

In [ ]:
log_evidence(Model1, N=2), log_evidence(Model2, N=2)

Roughly how precisely do we need to know the log Evidence, to be able to compare models? Run `log_evidence` with different values of `N` (the number of prior samples in the average) to until you're satisfied that you're getting a usefully accurate result for each model.

In [ ]:
print('Model 1:')
for Nevidence in [1, 10, 100, 1000, 10000]: # Do you need even more than this?
    %time logE1 = log_evidence(Model1, N=Nevidence)
    print("From", Nevidence, "samples, the log-evidence is", logE1)

**Self-check:** For the exponential model, I converge to a log-evidence of $\approx -837$.

In [ ]:
print('Model 2:')
for Nevidence in [1, 10, 100, 1000, 10000]: # Do you need even more than this?
    %time logE2 = log_evidence(Model2, N=Nevidence)
    print("From", Nevidence, "samples, the log-evidence is", logE2)

So, we have log evidences computed for each model. Now what? We just compare their difference to the Jeffreys scale again:

In [ ]:
logE2 - logE1

Note that we might end up with a different conclusion as to the strength of any preference of Model 2, compared with the DIC! The reason is that the evidence explicitly accounts for the information in the prior (which, recall, counts as part of the model definition), while the DIC does this much less directly.

We could also be good Bayesians and admit that there should be a prior distribution in model space. For example, maybe I have a very compelling theoretical reason why the offset distribution should be exponential (I don't, but just for example). Then, I might need some extra convincing that an alternative model is required.

We would then compute the ratio of the posterior probabilities of the models as follows:

In [ ]:
prior_H1 = 0.9 # or your choice
prior_H2 = 1.0 - prior_H1 # assuming only these two options

log_post_H1 = np.log(prior_H1) + logE1
log_post_H2 = np.log(prior_H2) + logE2

print('Difference of log posteriors (H2-H1):', log_post_H2-log_post_H1)
print('Ratio of posteriors (H2/H1):', np.exp(log_post_H2-log_post_H1)) # NB this one might over/underflow

Depending on the fitness of the alternative model you chose, you may find that only an extremely lopsided prior in model space would influence your conclusion.

Comment on what you find from the evidence, compared with the DIC.

> TBC: _your answer_

## 3. Compute the evidence with `dynesty`

To get some experience with a package that uses nested sampling to compute the evidence, let's repeat Section 2 using `dynesty`.

Looking at [the docs](https://dynesty.readthedocs.io/en/latest/crashcourse.html), we first need a function that maps the unit cube onto our prior. For a uniform prior, this is a simple translation and rescaling, but for other priors it involves going through the prior distribution's quantile function. Fortunately, we assumed uniform priors for Model 1! However, if you followed the advice in the previous notebook to keep a dictionary of univariate priors in the `Model` object, and those priors are `scipy.stats` distributions, it's relatively easy to handle this slightly more general case. However you choose to do it, implement a function that performs this transformation for the 1-parameter Exponential model (and priors) below.

In [ ]:
def ptform(u):
    '''
    Input: a vector in the unit cube, 0 <= u[i] <= 1, that conforms to the prior volume.
    Output: a vector in our parameter space.
    '''
    TBC()
    
TBC_above()

Do a sanity check - the cell below should return the quartiles of the uniform prior.

In [ ]:
print(ptform([0.25]), ptform([0.5]), ptform([0.75]))

We'll also need a log-likelihood function that takes a vector of parameters as input, in contrast to how it's written in the Model class. This could be solved similarly to what's done in the `Model.log_posterior` method, or we could just define:

In [ ]:
def dyn_log_like(params):
    return Model1.log_likelihood(*params)

Then we can just go ahead and run.

In [ ]:
%%time
sampler = dynesty.NestedSampler(dyn_log_like, ptform, len(Model1.param_names))
sampler.run_nested()
results = sampler.results

And look at some stuff (see the Dynesty documentation for an explanation of what all this is):

In [ ]:
# Plot a summary of the run.
rfig, raxes = dynesty.plotting.runplot(results)

The Evidence panel appears to be utterly unhelpful in this case (at least when I ran it), so below is a plot of the log-evidence as a function of iteration, discarding the beginning when the accumulated evidence is truly tiny. It should show us converging to a value similar to what we found earlier.

In [ ]:
plt.rcParams['figure.figsize'] = (12.0, 4.0)
Ndiscard = 100
plt.plot(results['logz'][Ndiscard:]);
plt.xlabel("Iteration - "+str(Ndiscard), fontsize=14);
plt.ylabel("Log Evidence", fontsize=14);

This extracts the log-evidence at the last iteration, which may or not be the approved method.

In [ ]:
results['logz'][-1]

Let's compare with what we got from simple monte carlo:

In [ ]:
logE1 - results['logz'][-1]

That was so easy, let's go ahead and do the same for Model 2. Implement the transformation from the unit cube to your model's parameter space:

In [ ]:
def ptform2(u):
    '''
    Input: a vector in the unit cube, 0 <= u[i] <= 1, that conforms to the prior volume.
    Output: a vector in our parameter space.
    '''
    TBC()
    
TBC_above()

And run this elegant definition:

In [ ]:
def dyn_log_like2(params):
    return Model2.log_likelihood(*params)

Now we can go ahead and run the sample.

In [ ]:
%%time
sampler = dynesty.NestedSampler(dyn_log_like2, ptform2, len(Model2.param_names))
sampler.run_nested()
results = sampler.results

... and produce the same set of plots.

In [ ]:
rfig, raxes = dynesty.plotting.runplot(results)

In [ ]:
plt.rcParams['figure.figsize'] = (12.0, 4.0)
Ndiscard = 100
plt.plot(results['logz'][Ndiscard:]);
plt.xlabel("Iteration - "+str(Ndiscard), fontsize=14);
plt.ylabel("Log Evidence", fontsize=14);

Finally, extract the final evidence,

In [ ]:
results['logz'][-1]

... and compare it to the SMC result.

In [ ]:
logE2 - results['logz'][-1]

Given the work you'd already done, that was (I hope) staggeringly easy to set up, even if it did take longer to compute the same answer (I find) in this case. In larger-dimensional parameter spaces, it's easy to imagine this method being far preferable to blindly sampling from the prior. Note that we run `dynesty` out of the box here - there are many options (and some potential failure modes) that you should read about before using it in the real world.